# Lys2 Version 2 - sent to Bruce on Jan 27 
## try to minimize! the codon pair score in the cost function
## no longer care about keeping the rest of the gene wt
The minimum possible codon pair score for for [120:-60], without caring for any other sequences is -293.61

In [129]:
import csv
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sys        
from scipy import stats as scistats

# number of desired codon paris in sequence of any length
def num_costly_codon_pairs(seq, count_negatives):
    num = 0
    for i in range(0,(len(seq)-3),3):
        '''if (seq[i+2] == "T" and seq[i+3] == "A" and ((seq[i+4] == "A" or seq[i+4] == "G"))):
            num += 1
        if (count_negatives):
            if (seq[i+2] == "C" and seq[i+3] == "G"):
                num -= 1'''
        '''if (seq[i+2] == "C" and seq[i+3] == "G"):
            num += 1
        if (count_negatives):
            if (seq[i+2] == "T" and seq[i+3] == "A" and ((seq[i+4] == "A" or seq[i+4] == "G"))):
                num -= 1'''
        if (seq[i+2] == "T" and seq[i+3] == "A" and ((seq[i+4] == "T" or seq[i+4] == "C"))):
            num += 1
        if (count_negatives):
            if (seq[i+2] == "C" and seq[i+3] == "G"):
                num -= 1   
    return(num)

# codon pair score of any length sequence
def get_cps(seq):
    cps = 0
    if (len(seq) == 6):
        return(cp_scores[seq])  
    for i in range(0,(len(seq)-6),3):
        cps += cp_scores[seq[i:i+6]]
    return(cps)

# codon adaptation index of any length sequence
def get_cai(seq):
    codon_occs = dict()
    # set up the dict
    for codon in (codon_to_aa_dict.keys()):
        codon_occs[codon] = 0
    # get the frequency of each codon
    for i in range(0,len(seq),3):
        codon_occs[seq[i:(i+3)]] += 1
    # get the CAI of each codon
    codon_CAI = dict()
    for aa in (aa_to_syn_codons_dict.keys()):
        highest_frequency = 0
        for codon in (aa_to_syn_codons_dict[aa]):
            if (codon_occs[codon] > highest_frequency):
                highest_frequency = codon_occs[codon]
        for codon in (aa_to_syn_codons_dict[aa]):
            if (highest_frequency == 0):
                codon_CAI[codon] = 0
            else:
                codon_CAI[codon] = codon_occs[codon]/highest_frequency
    # get the CAI as the geometric mean of all the codon's individual CAI values
    cai_list = []
    for i in range(0,len(seq),3):
        cai_list.append(codon_CAI[seq[i:(i+3)]])
    return(scistats.gmean(cai_list))

# read in synonymous codons
aa_to_syn_codons_dict = dict()
codon_to_aa_dict = dict()
csvfile = open('AA_CODONS_MAP')
reader = csv.reader(csvfile, delimiter='\t')
for row in reader:
    mylist = row[3].split(',')
    aa_to_syn_codons_dict[row[0]] = mylist
    for codon in mylist:
        codon_to_aa_dict[codon] = row[0]

# read in the codon pair scores
cp_scores = dict()
cp_scores_file = open("yeast_cps.csv")
cp_reader = csv.reader(cp_scores_file, delimiter=',')
for line in cp_reader:
    cp_scores[line[0]] = float(line[1])

# read in the gene's codon sequence
codons_file = open("S288C_YBR115C_LYS2_genomic.fsa")
original_codons = ""
for line in codons_file:
    line = line.rstrip()
    if line[0] != ">":
        original_codons = original_codons + line
codons_file.close()

# read in the gene's amino acid sequence
aa_file = open("S288C_YBR115C_LYS2_protein.fsa")
original_AA = ""
for l in aa_file:
    l = l.rstrip()
    if l[0] != ">":
        original_AA = original_AA + l
aa_file.close()

if (len(original_AA)*3 != len(original_codons)):
    print("PROBLEM: codons and AA don't match")

    
# store the sequences, and then work with the ones that have "chopped off" start and end
long_original_AA = original_AA
original_AA = original_AA[40:-20]
long_original_codons = original_codons
original_codons = original_codons[120:-60]
   
# store_best will contain, at each AA position, and at each of the syn codon choices,
# the best cost of the suffix that starts with this codon, the current codon pair score,
# and the next codon that corresponds to this cost
store_best = [None] * len(original_AA)
for i in range(0,len(original_AA)):
    store_best[i] = dict()
    for codon in aa_to_syn_codons_dict[original_AA[i]]:
        store_best[i][codon] = 0
                

# initialize the dynamic program by setting store_best for the last amino acid position
for codon in store_best[-1]:
    store_best[i][codon] = (0,0,"")


# do the dynamic program, stating from the one before the end of the list, until the beginning
for pos in reversed(range(0, len(store_best)-1)):
    for codon in store_best[pos]:
        # consider as cost the cost of adding this codon to one of the best sequences
        # from the previous suffix
        best_prev = ""
        best_cps = sys.maxsize # keep this
        best_cost = -1 #sys.maxsize #-1
        for prev_codon in store_best[pos+1]:
            cost = num_costly_codon_pairs(codon+prev_codon,1) + store_best[pos+1][prev_codon][0]
            cp_score = get_cps(codon+prev_codon) + store_best[pos+1][prev_codon][1]
            #print(cp_score)
            # maximizer the cost
            #if (cp_score < best_cps):
            if ((cost > best_cost) or # > <
                # break the tie with lowest codon pair score
                ((cost == best_cost) and (cp_score < best_cps))):
                best_cost = cost
                best_cps = cp_score
                best_prev = prev_codon
        store_best[pos][codon] = (best_cost, best_cps, best_prev)
        #print("storing for " + str(pos) + " " + codon + ": " + str(best_cps) + " " + best_prev)

# going from the beginning to the end, read off the recoded sequences from the store_best
# re-doing the computation for the first position, to pick the best of the syn codons
# the rest of the seuqnces is just read out
recoded_sequence = ""
next_codon = ""
for pos in range(0, len(store_best)):
    if pos == 0:
        # maximum
        best_first = ""
        best_first_cps = sys.maxsize # keep this
        best_first_cost = -1 # sys.maxsize #-1
        for codon in store_best[0]:
            # maximize the cost
            #if (store_best[pos][codon][1] < best_first_cps):
            if ((store_best[pos][codon][0] > best_first_cost) or  #> <
               ((store_best[pos][codon][0] == best_first_cost) and (store_best[pos][codon][1] < best_first_cps))):
                best_first = codon
                best_first_cost = store_best[pos][codon][0]
                best_first_cps = store_best[pos][codon][1]
                next_codon = store_best[pos][codon][2]
        recoded_sequence += best_first
    else:
        recoded_sequence += next_codon
        next_codon = store_best[pos][next_codon][2]

# now restore the original sequences and patch up the recoded sequence with the beginning and end
original_AA = long_original_AA
original_codons = long_original_codons
recoded_sequence = original_codons[0:120] + recoded_sequence + original_codons[-60:]

#print(original_codons)
print(recoded_sequence)    
back_to_original = ""
diff_total = 0
for i in range(0,len(recoded_sequence),3):
    if (recoded_sequence[i:i+3] != original_codons[i:i+3]):
        diff_total += 1
    back_to_original += codon_to_aa_dict[recoded_sequence[i:i+3]]
print("total number of codon diferences between original and recoded sequences " + str(diff_total))
if (original_AA == back_to_original):
    print("AA sequence remains the same, good")
print(len(original_codons))
print("num desired codon pairs in original sequence: " + str(num_costly_codon_pairs(original_codons, 0)))
print("num desired codon pairs in recoded sequence: " + str(num_costly_codon_pairs(recoded_sequence, 0)))
print("codon pair score of the original sequence: " + str(get_cps(original_codons)))
print("codon pair score of the recoded sequence: " + str(get_cps(recoded_sequence)))
print("CAI of the original sequence is: " + str(get_cai(original_codons)))
print("CAI of the recoded sequence is: " + str(get_cai(recoded_sequence)))

ATGACTAACGAAAAGGTCTGGATAGAGAAGTTGGATAATCCAACTCTTTCAGTGTTACCACATGACTTTTTACGCCCACAACAAGAACCTTATACGAAACAAGCTACATATTCGTTACAGCTCCCACAACTTGATGTCCCGCATGATAGCTTCTCTAATAAGTATGCAGTTGCACTTAGTGTGTGGGCAGCACTTATCTATCGGGTTACAGGGGATGATGATATAGTCTTGTATATAGCTAATAATAAGATACTTAGATTTAATATACAACCTACTTGGAGCTTTAATGAATTGTATAGTACTATTAATAATGAACTTAATAAGCTTAATTCTATAGAAGCTAACTTCTCTTTTGATGAATTAGCTGAGAAGATACAATCCTGTCAGGATCTAGAACGTACACCCCAATTGTTTCGTCTGGCTTTTCTTGAGAATCAAGACTTTAAGTTAGATGAATTTAAACACCACCTAGTTGACTTTGCACTTAATCTAGATACGTCTAATAATGCCCATGTACTTAATCTTATCTATAATAGCTTGTTGTATAGTAATGAGCGGGTTACTATAGTTGCGGATCAATTTACACAATACCTTACAGCAGCCTTGTCGGACCCGTCTAATTGTATTACTAAGATTAGCCTTATTACAGCCTCGTCTAAAGATAGCCTCCCAGACCCTACTAAGAATCTAGGTTGGTGTGACTTTGTAGGTTGTATACATGATATCTTTCAGGATAATGCTGAGGCTTTTCCAGAACGTACTTGTGTAGTTGAGACCCCTACACTTAATTCAGATAAGTCTAGATCTTTTACATATCGGGATATTAATCGTACGTCTAATATAGTAGCCCACTACCTTATAAAAACGGGTATTAAGCGGGGGGATGTAGTTATGATCTATAGCTCCCGGGGGGTTGATCTTATGGTGTGTGTTATGGGAGTCCTTAAAGCGGGGGCTACATTCTCAGTTATAGATCCAGCCTACCCCCCGGCCCGACAGA

# Version 1
vienna package - something like mfold - look into with justin - Bruce not worried about this
vienna package verbose

splice sites - Justin,
https://users.soe.ucsc.edu/~leslie/runIntronSearch.html
http://harlequin.jax.org/drupal/?q=yeastRNApred

transcriptional terminators - Justin did not do anything;  they are not well defined....

restriction sites - Bruce (look at what genscript uses)

In [87]:
import csv
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import sys

aa_to_syn_codons_dict = dict()
codon_to_aa_dict = dict()
csvfile = open('AA_CODONS_MAP')
reader = csv.reader(csvfile, delimiter='\t')
for row in reader:
    mylist = row[3].split(',')
    aa_to_syn_codons_dict[row[0]] = mylist
    for codon in mylist:
        codon_to_aa_dict[codon] = row[0]
        
def get_cps(seq):
    cps = 0
    for i in range(0,(len(seq)-6),3):
        cps += cp_scores[seq[i:i+6]]
    return(cps)        

def num_costly_codon_pairs(seq, do_print):
    num = 0
    to_print = ""
    for i in range(0,(len(seq)-3),3):
        if (seq[i+2] == "C" and seq[i+3] == "G"):
        #if (seq[i+2] == "T" and seq[i+3] == "A" and ((seq[i+4] == "A" or seq[i+4] == "G"))):
            num += 1
            if (do_print == 1):
                to_print += str(i) + " " #seq[i:i+6] + " " + 
    if (do_print):
        print(to_print)
    return(num)

cp_scores = dict()
cp_scores_file = open("yeast_cps.csv")
cp_reader = csv.reader(cp_scores_file, delimiter=',')
for line in cp_reader:
    cp_scores[line[0]] = float(line[1])
    
codons_file = open("S288C_YBR115C_LYS2_genomic.fsa")
original_codons = ""
for line in codons_file:
    line = line.rstrip()
    if line[0] != ">":
        original_codons = original_codons + line
codons_file.close()

aa_file = open("S288C_YBR115C_LYS2_protein.fsa")
original_AA = ""
for l in aa_file:
    l = l.rstrip()
    if l[0] != ">":
        original_AA = original_AA + l
aa_file.close()

if (len(original_AA)*3 != len(original_codons)):
    print("PROBLEM: codons and AA don't match")

    
# store the sequences, and then work with the ones that have "chopped off" start and end
long_original_AA = original_AA
original_AA = original_AA[40:-20]
long_original_codons = original_codons
original_codons = original_codons[120:-60]
    
store_best = [None] * len(original_AA)
for i in range(0,len(original_AA)):
    store_best[i] = dict()
    for codon in aa_to_syn_codons_dict[original_AA[i]]:
        store_best[i][codon] = 0
                

# do the last AA
for codon in store_best[-1]:
    store_best[i][codon] = (0,"")


# do the dynamic program, stating from the one before the end of the list, until the end
for pos in reversed(range(0, len(store_best)-1)):
    for codon in store_best[pos]:
        # consider as cost the cost of adding this codon to one of the best sequences
        # from the previous suffix
        # maximizer the dimers
        best_prev = ""
        best_cost = -1 #sys.maxsize #-1
        for prev_codon in store_best[pos+1]:
            cost = num_costly_codon_pairs(codon+prev_codon,0) + store_best[pos+1][prev_codon][0]
            # maximizer the dimers
            if ((cost > best_cost) or # > <
                ((cost == best_cost) and (prev_codon == original_codons[(pos+1)*3:(pos+1)*3+3]))): #try to keep same seq
                best_cost = cost
                best_prev = prev_codon
        store_best[pos][codon] = (best_cost, best_prev)
        #print("storing for " + str(pos) + " " + codon + ": " + str(best_cost) + " " + best_prev)

recoded_sequence = ""
next_codon = ""
for pos in range(0, len(store_best)):
    if pos == 0:
        # maximum
        best_first = ""
        best_first_cost = -1 # sys.maxsize #-1
        for codon in store_best[0]:
            # maximum
            if ((store_best[pos][codon][0] > best_first_cost) or  #> <
                ((cost == best_cost) and (codon == original_codons[(pos+1)*3:(pos+1)*3+3]))): #try to keep same seq
                best_first = codon
                best_first_cost = store_best[pos][codon][0]
                next_codon = store_best[pos][codon][1]
        recoded_sequence += best_first
    else:
        recoded_sequence += next_codon
        next_codon = store_best[pos][next_codon][1]

# now restore the original sequences and patch up the recoded sequence with the beginning and end
original_AA = long_original_AA
original_codons = long_original_codons
recoded_sequence = original_codons[0:120] + recoded_sequence + original_codons[-60:]

#print(original_codons)
print(recoded_sequence)    
print("num costly codon pairs: " + str(num_costly_codon_pairs(recoded_sequence, 0)))
back_to_original = ""
diff_total = 0
for i in range(0,len(recoded_sequence),3):
    if (recoded_sequence[i:i+3] != original_codons[i:i+3]):
        diff_total += 1
    back_to_original += codon_to_aa_dict[recoded_sequence[i:i+3]]
print("total number of codon diferences between original and recoded sequences " + str(diff_total))
if (original_AA == back_to_original):
    print("AA sequence remains the same, good")
print(len(original_codons))
print("num costly codon pairs in original sequences: " + str(num_costly_codon_pairs(original_codons, 0)))
print("codon pair score of the original sequence: " + str(get_cps(original_codons)))
print("codon pair score of the recoded sequence: " + str(get_cps(recoded_sequence)))

ATGACTAACGAAAAGGTCTGGATAGAGAAGTTGGATAATCCAACTCTTTCAGTGTTACCACATGACTTTTTACGCCCACAACAAGAACCTTATACGAAACAAGCTACATATTCGTTACAGTTACCTCAGCTCGACGTGCCTCACGATAGTTTTTCTAACAAATACGCCGTCGCTTTGAGCGTATGGGCCGCATTGATATATCGCGTAACCGGCGACGACGATATCGTTCTTTATATCGCGAATAACAAAATCTTAAGATTCAATATTCAACCAACGTGGTCATTTAACGAGCTGTATTCTACAATTAACAACGAGTTGAACAAGCTCAATTCTATCGAGGCCAATTTTTCCTTCGACGAGCTCGCCGAAAAAATTCAAAGTTGCCAAGATCTCGAAAGGACCCCTCAGTTGTTCCGTCTCGCCTTTCTCGAAAACCAAGATTTCAAACTCGACGAGTTCAAGCATCATCTCGTCGACTTCGCTTTGAATCTCGATACCAGTAATAACGCGCACGTTTTGAACTTAATTTATAACAGCTTACTGTATTCGAACGAACGCGTAACCATCGTCGCCGACCAATTTACTCAATATTTGACCGCCGCGCTAAGCGATCCATCCAATTGCATAACTAAAATCTCTCTGATCACCGCATCATCCAAGGATAGTTTACCCGATCCAACTAAGAACCTCGGCTGGTGCGATTTCGTCGGGTGTATTCACGACATTTTCCAGGACAACGCCGAAGCCTTCCCCGAGAGAACCTGCGTCGTCGAGACTCCAACACTAAATTCCGACAAGTCCCGTTCTTTCACTTATCGCGACATCAACCGCACTTCTAACATCGTCGCCCATTATTTGATTAAAACCGGTATCAAACGCGGCGACGTCGTGATGATCTATTCTTCTCGCGGCGTCGATTTGATGGTATGCGTGATGGGCGTCTTGAAAGCCGGCGCAACCTTTAGCGTTATCGACCCCGCATATCCCCCCGCCAGACAAA